# Collaborative filtering

- Coger subsets aleatorios
- Ver si usar item based collab porque tiene mas escalabilidad, user based necesita calcular la similitud de todos los pares de usuarios

## Data loading

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#df_users = pd.read_csv('data/train_triplets.txt', sep='\t', header=None, names=['user_id', 'song_id', 'play_count'])
df_users = pd.read_csv('data/Million Song Dataset kaggle/User Listening History.csv')

In [3]:
df_music = pd.read_csv('data/Million Song Dataset kaggle/Music Info.csv')

In [4]:
df_users

,track_id,user_id,playcount
0,TRIRLYL128F42539D1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
1,TRFUPBA128F934F7E1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
2,TRLQPQJ128F42AA94F,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
3,TRTUCUY128F92E1D24,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
4,TRHDDQG12903CB53EE,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
...,...,...,...
9711296,TRBKFKL128E078ED76,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,1
9711297,TRYFXPG128E078ECBD,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,1
9711298,TROBUUZ128F4263002,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,1
9711299,TROEWXC128F148C83E,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,1


In [5]:
df_users.dtypes

track_id     object
user_id      object
playcount     int64
dtype: object

In [6]:
num_distinct_users = df_users['user_id'].nunique()
num_distinct_users

962037

In [7]:
data_df = df_users.groupby('track_id').size().reset_index(name='count')

In [8]:
data_df

,track_id,count
0,TRAAAED128E0783FAB,161
1,TRAAAHD128F42635A5,2
2,TRAAAQO12903CD8E1C,1
3,TRAABJS128F9325C99,945
4,TRAABXA128F9326351,18
...,...,...
30454,TRZZYMU128E0792400,1140
30455,TRZZZCL128F428BB80,19
30456,TRZZZCN128F9317A03,15
30457,TRZZZHL128F9329CFB,116


In [9]:
df_music.head()

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,7,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4


In [10]:
df_music_info = df_music[['track_id', 'name', 'artist', 'energy']]
df_music_info.head()

,track_id,name,artist,energy
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,0.918
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,0.892
2,TROUVHL128F426C441,Come as You Are,Nirvana,0.826
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,0.664
4,TRLNZBD128F935E4D8,Creep,Radiohead,0.430


In [11]:
df_music_count = df_music_info.merge(
    df_users.groupby('track_id').size().reset_index(name='playcount'),
    on='track_id',
    how='left'
)
df_music_count = df_music_count.fillna(0)
print(df_music_count.shape)
df_music_count.head()

(50683, 5)


,track_id,name,artist,energy,playcount
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,0.918,41.0
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,0.892,378.0
2,TROUVHL128F426C441,Come as You Are,Nirvana,0.826,0.0
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,0.664,20.0
4,TRLNZBD128F935E4D8,Creep,Radiohead,0.430,0.0


In [12]:
# df_music_count = df_music_info.merge(
#     df_users.groupby('track_id').size().reset_index(name='playcount'),
#     on='track_id',
#     how='inner'
# )
# print(df_music_count.shape)
# df_music_count.head()

## Interaction Matrix

In [13]:
df_users_agg = df_users.groupby('user_id')['playcount'].agg(
    #total_playcount='sum',
    max_playcount='max'
).reset_index()
df_users_agg = df_users_agg.rename(columns={'playcount': 'max_playcount'})

In [14]:
df_users_rating = df_users.merge(df_users_agg, on='user_id')

We create 2 interaction matrix
- Binary version: 1 indicates a user has interacted with the song and 0 indicates he has not
- Weighted version: Rating in range (0,1], calculated as (user playcount) / (playcount of user's most listened song). Gives more rating to those more listened by a user

In [15]:
df_users_rating['rating'] = df_users_rating['playcount'] / df_users_rating['max_playcount']
df_users_rating[:50]

,track_id,user_id,playcount,max_playcount,rating
0,TRIRLYL128F42539D1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.500000
1,TRFUPBA128F934F7E1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.500000
2,TRLQPQJ128F42AA94F,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.500000
3,TRTUCUY128F92E1D24,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.500000
4,TRHDDQG12903CB53EE,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.500000
5,TRGRDEC128F423C07D,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.500000
6,TRRYCBO128F932A2C7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.500000
7,TRUTULC128F4293712,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.500000
8,TRAAHSY128F147BB5C,b80344d063b5ccb3212f76538f3d9e43d87dca9e,2,2,1.000000
9,TRDLMWP128F426BF6C,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.500000


In [16]:
df_users_agg[df_users_agg['user_id'] == '85c1f87fea955d09b4bec2e36aee110927aedf9a']

,user_id,max_playcount
502464,85c1f87fea955d09b4bec2e36aee110927aedf9a,3


For the interaction matrix, we shall use a sparse matrix, due to users only listening to a small subset of the songs

In [17]:
from scipy.sparse import csr_matrix, csc_matrix

We cant reproduce next line because it needs 218 GB of RAM to create the table

In [18]:
#df_music_pivoted = df_users.pivot(index='user_id', columns='track_id', values='rating').fillna(0)

We codify each user and song to a unique integer ID

In [19]:
user_codes, user_uniques = pd.factorize(df_users['user_id'])
track_codes, track_uniques = pd.factorize(df_users['track_id'])

We create the interaction matrix as a sparse csr matrix for user-based collaborative filtering and as csc for item-based collaborative filtering

In [20]:
interaction_matrix_csr_binary = csr_matrix((np.ones(len(df_users)), (user_codes, track_codes)),shape=(len(user_uniques), len(track_uniques)))
interaction_matrix_csr_weighted = csr_matrix((df_users_rating['rating'], (user_codes, track_codes)),shape=(len(user_uniques), len(track_uniques)))

In [21]:
interaction_matrix_csc_binary = csc_matrix((np.ones(len(df_users)), (user_codes, track_codes)),shape=(len(user_uniques), len(track_uniques)))
interaction_matrix_csc_weighted = csc_matrix((df_users_rating['rating'], (user_codes, track_codes)),shape=(len(user_uniques), len(track_uniques)))

In [22]:
interaction_matrix_csc_weighted.shape

(962037, 30459)

In [23]:
# Calcular el espacio en memoria ocupado por interaction_matrix en MB
memory_usage_bytes = (
    interaction_matrix_csc_weighted.data.nbytes +
    interaction_matrix_csc_weighted.indptr.nbytes +
    interaction_matrix_csc_weighted.indices.nbytes
)
memory_usage_mb = memory_usage_bytes / (1024 ** 2)
print(f"Espacio en memoria de interaction_matrix: {memory_usage_mb:.2f} MB")

Espacio en memoria de interaction_matrix: 111.25 MB


The sparse matrix has a size of only 114.81 MB, while the normal matrix as a dataframe was 218 GB

In [24]:
interaction_matrix_csc_binary[:10, :30].todense()

matrix([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.

In [25]:
interaction_matrix_csc_weighted[:10, :30].todense()

matrix([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
         0.5       , 0.5       , 0.5       , 1.        , 0.5       ,
         0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
         0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.66666667, 1.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ,

2 aproximations:
- Filter songs by energy and recomend
- Recomend and chose song of recomended list based on energy

We discard user based because we would need to perform similarity metrics every single time, what is not scalable to a system with a huge amount of users

## Item based collaborative filtering

We use cosinge similarity as a similarity metric

In [40]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csc_matrix

# Calculate item-based (track-based) cosine similarity
# Transpose interaction_matrix_weighted to get items as rows
item_based_similarity = cosine_similarity(interaction_matrix_csc_weighted.T, dense_output=False)
item_based_similarity = csc_matrix(item_based_similarity)

In [41]:
memory_usage_bytes = (
    item_based_similarity.data.nbytes +
    item_based_similarity.indptr.nbytes +
    item_based_similarity.indices.nbytes
)
memory_usage_mb = memory_usage_bytes / (1024 ** 2)
print(f"Espacio en memoria de item_based_similarity: {memory_usage_mb:.2f} MB")

Espacio en memoria de item_based_similarity: 939.41 MB


Solo 7 segundos de ejecucion

In [27]:
item_based_similarity.shape

(30459, 30459)

In [28]:
class ItemBasedRecommender:
    def __init__(self, interaction_matrix, item_similarity, track_uniques, df_music_info):
        self.interaction_matrix = interaction_matrix
        self.item_similarity = item_similarity
        self.track_uniques = track_uniques
        self.df_music_info = df_music_info
        self.user_index = None
        self.recommendations = None # List of tuples (track_id, energy, has been recommended)

    def make_recommendations(self, user_index, n=100):
        self.user_index = user_index
        user_ratings = self.interaction_matrix[self.user_index]
        interacted = user_ratings.nonzero()[1]
        scores = self.item_similarity.dot(user_ratings.T).toarray().ravel()
        scores[interacted] = -np.inf
        top_n_index = np.argpartition(scores, -n)[-n:]
        top_n_index = top_n_index[np.argsort(scores[top_n_index])[::-1]]
        track_ids = self.track_uniques[top_n_index]
        df_filtered = self.df_music_info[self.df_music_info['track_id'].isin(track_ids)][['track_id', 'energy']]

        self.recommendations = [(track_id, energy, False) for track_id, energy in df_filtered.itertuples(index=False, name=None)]

    
    def recommend_song(self, energy, energy_margin=0.05):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        
        closest_track_index = None
        distance_to_energy = float('inf')

        for i, (track_id, track_energy, has_been_recommended) in enumerate(self.recommendations):
            distance = abs(track_energy - energy)

            if not has_been_recommended and distance <= energy_margin:
                self.recommendations[i] = (track_id, track_energy, True)
                return (track_id, track_energy)
            
            if not has_been_recommended and distance < distance_to_energy:
                closest_track_index = i
                distance_to_energy = distance
        
        if closest_track_index is not None:
            track_id, track_energy, _ = self.recommendations[closest_track_index]
            self.recommendations[closest_track_index] = (track_id, track_energy, True)
            return (track_id, track_energy)

        raise ValueError("All recommendations have already been recommended")


    def get_recommendations(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return self.recommendations


    def get_recommendations_ids(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return [track_id for track_id, _, _ in self.recommendations]
    
    
    def get_recommendations_info(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return self.df_music_info[self.df_music_info['track_id'].isin([track_id for track_id, _, _ in self.recommendations])]

### Example

In [29]:
user_index = 0 #User for wich recommendations will be made

Songs the user has listened to

In [30]:
user_listened_songs = df_users[df_users['user_id'] == user_uniques[user_index]].track_id
df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

,track_id,name,artist,energy
79,TRAAHSY128F147BB5C,Speed of Sound,Coldplay,0.8980
796,TRDLMWP128F426BF6C,Ragged Wood,Fleet Foxes,0.6850
1158,TRPUGUW128F426BF6F,He Doesn't Know Why,Fleet Foxes,0.5580
1743,TRRYCBO128F932A2C7,Love Shack,The B-52's,0.9020
2350,TRFUPBA128F934F7E1,Paper Gangsta,Lady Gaga,0.5300
2585,TRVODUZ128F934D094,All That We Perceive,Thievery Corporation,0.8340
2722,TRHDDQG12903CB53EE,Heaven's Gonna Burn Your Eyes,Thievery Corporation,0.3370
5416,TRTUCUY128F92E1D24,Stacked Actors,Foo Fighters,0.9340
5428,TRLXSNR128F429361D,Generator,Foo Fighters,0.9240
5819,TRADVZX128F426BF79,Sun Giant,Fleet Foxes,0.0626


In [76]:
item_based_recommender = ItemBasedRecommender(interaction_matrix_csc_weighted, item_based_similarity, track_uniques, df_music_info)
item_based_recommender.make_recommendations(user_index, n=100)

In [77]:
recommended_song = item_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
2387,TRMYAYJ128F934D0AF,Until the Morning,Thievery Corporation,0.432


In [78]:
recommended_song = item_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
8610,TRBIVWU128F92CA9D2,Skin and Bones,Foo Fighters,0.396


In [79]:
recommended_song = item_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
9054,TRSLJOC128F42889D5,The Sharing Song,Jack Johnson,0.427


In [80]:
item_based_recommender.get_recommendations_info().head(10)

,track_id,name,artist,energy
550,TRZJGCC128F427A496,For Emma,Bon Iver,0.613
847,TRQLPHT128F427A495,Team,Bon Iver,0.784
860,TRGEIDA128F933B4B8,Tiger Mountain Peasant Song,Fleet Foxes,0.146
1109,TRVRIBZ128F426BF71,Your Protector,Fleet Foxes,0.537
1117,TRLPCKY128F42ACFB8,DOA,Foo Fighters,0.914
1199,TRQEMRN128F933B4B2,Sun It Rises,Fleet Foxes,0.275
1270,TRDJRES128F933B4BA,Quiet Houses,Fleet Foxes,0.668
1272,TRKOHVA128F426BF70,Heard Them Stirring,Fleet Foxes,0.470
1317,TRVQTQG128F93073B0,Molly's Chambers,Kings of Leon,0.920
1340,TRWAUCC128F428E6FD,Staple It Together,Jack Johnson,0.794


## User based collaborative filtering

In [37]:
user_similarity = cosine_similarity(interaction_matrix_csr_weighted, dense_output=False)

MemoryError: Unable to allocate 240. GiB for an array with shape (32266149955,) and data type int64

Too big, is not viable. We must use a more efficient aproach: knn

In [42]:
from sklearn.neighbors import NearestNeighbors

In [71]:
class UserBasedRecommender:
    def __init__(self, interaction_matrix, track_uniques, df_music_info, num_neighbors=10):
        self.interaction_matrix = interaction_matrix
        self.knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=num_neighbors, n_jobs=-1)
        self.knn.fit(interaction_matrix)
        self.track_uniques = track_uniques
        self.df_music_info = df_music_info
        self.user_index = None
        self.recommendations = None # List of tuples (track_id, energy, has been recommended)

    def make_recommendations(self, user_index, n=5):
        self.user_index = user_index

        distances, neighbors_indices = self.knn.kneighbors(self.interaction_matrix[user_index])

        neighbors_interaction_matrix = self.interaction_matrix[neighbors_indices.flatten()]

        neighbors_mean_ratings = neighbors_interaction_matrix.mean(axis=0).A1 # Converts to 1D array

        user_ratings = self.interaction_matrix[user_index]
        interacted = user_ratings.nonzero()[1]
        #neighbors_mean_ratings[interacted] = -np.inf #Asign -np.inf to already listened songs by the user

        top_n_index = np.argpartition(neighbors_mean_ratings, -n)[-n:]
        top_n_index = top_n_index[np.argsort(neighbors_mean_ratings[top_n_index])[::-1]]
        track_ids = self.track_uniques[top_n_index]

        df_filtered = self.df_music_info[self.df_music_info['track_id'].isin(track_ids)][['track_id', 'energy']]

        self.recommendations = [(track_id, energy, False) for track_id, energy in df_filtered.itertuples(index=False, name=None)]

    
    def recommend_song(self, energy, energy_margin=0.05):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        
        closest_track_index = None
        distance_to_energy = float('inf')

        for i, (track_id, track_energy, has_been_recommended) in enumerate(self.recommendations):
            distance = abs(track_energy - energy)

            if not has_been_recommended and distance <= energy_margin:
                self.recommendations[i] = (track_id, track_energy, True)
                return (track_id, track_energy)
            
            if not has_been_recommended and distance < distance_to_energy:
                closest_track_index = i
                distance_to_energy = distance
        
        if closest_track_index is not None:
            track_id, track_energy, _ = self.recommendations[closest_track_index]
            self.recommendations[closest_track_index] = (track_id, track_energy, True)
            return (track_id, track_energy)

        raise ValueError("All recommendations have already been recommended")


    def get_recommendations(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return self.recommendations


    def get_recommendations_ids(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return [track_id for track_id, _, _ in self.recommendations]
    
    
    def get_recommendations_info(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return self.df_music_info[self.df_music_info['track_id'].isin([track_id for track_id, _, _ in self.recommendations])]

## Example

In [81]:
user_index = 0 #User for wich recommendations will be made

In [82]:
user_listened_songs = df_users[df_users['user_id'] == user_uniques[user_index]].track_id
df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

,track_id,name,artist,energy
79,TRAAHSY128F147BB5C,Speed of Sound,Coldplay,0.8980
796,TRDLMWP128F426BF6C,Ragged Wood,Fleet Foxes,0.6850
1158,TRPUGUW128F426BF6F,He Doesn't Know Why,Fleet Foxes,0.5580
1743,TRRYCBO128F932A2C7,Love Shack,The B-52's,0.9020
2350,TRFUPBA128F934F7E1,Paper Gangsta,Lady Gaga,0.5300
2585,TRVODUZ128F934D094,All That We Perceive,Thievery Corporation,0.8340
2722,TRHDDQG12903CB53EE,Heaven's Gonna Burn Your Eyes,Thievery Corporation,0.3370
5416,TRTUCUY128F92E1D24,Stacked Actors,Foo Fighters,0.9340
5428,TRLXSNR128F429361D,Generator,Foo Fighters,0.9240
5819,TRADVZX128F426BF79,Sun Giant,Fleet Foxes,0.0626


In [105]:
user_based_recommender = UserBasedRecommender(interaction_matrix_csc_weighted, track_uniques, df_music_info, num_neighbors=900000)
user_based_recommender.make_recommendations(user_index, n=100)

In [106]:
recommended_song = user_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
43,TRNNGMK128F423F034,Where Is My Mind?,Pixies,0.435


In [107]:
recommended_song = user_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
798,TRAWRKT128E0788857,Swing Life Away,Rise Against,0.437


In [108]:
recommended_song = user_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
1037,TRBVNWT128F93173BA,Help I'm Alive,Metric,0.381


In [109]:
user_based_recommender.get_recommendations_info().head(10)

,track_id,name,artist,energy
43,TRNNGMK128F423F034,Where Is My Mind?,Pixies,0.435
44,TRAALAH128E078234A,Bitter Sweet Symphony,The Verve,0.907
59,TRPFYYL128F92F7144,Float On,Modest Mouse,0.893
67,TRFWGOJ128E0780C8B,In My Place,Coldplay,0.566
73,TRCRCBT128F4260DD1,Such Great Heights,The Postal Service,0.814
95,TRWQINA128F9339E57,Buddy Holly,Weezer,0.920
98,TRFOVTO128F4228CC3,Fortunate Son,Creedence Clearwater Revival,0.761
123,TRBNYBX128F422EC61,Hey There Delilah,Plain White T's,0.261
135,TRARTEH128F423DBC1,15 Step,Radiohead,0.853
153,TRWVOJJ12903CCC654,My Immortal,Evanescence,0.285
